# 构建语音识别系统 - 解码与评测

## 如何从模型输出到识别文本

模型输出的结果为 (b, len, vocab_size)

### greedy search

每一步选取预测概率最大的词

In [1]:
import torch
def greedy_search(ctc_probs: torch.tensor, encoder_out_lens: torch.tensor):
    batch_size, maxlen = ctc_probs.size()[:2]
    topk_prob, topk_index = ctc_probs.topk(1, dim=2)
    topk_index = topk_index.view(batch_size, maxlen)
    encoder_out_lens = encoder_out_lens.view(-1).tolist()

    hyps = []

    for i in range(len(encoder_out_lens)):
        hyp = topk_index[i, :encoder_out_lens[i]].tolist()
        hyps.append(hyp)

    return hyps

In [2]:
tensordict = torch.load("./example2.pt")

pre = tensordict["pre"].to("cpu")
lens = tensordict["lens"].to("cpu")

print(lens)
res = greedy_search(pre, lens)

tensor([46, 51, 44, 44, 41, 49, 48, 48, 74, 93, 44, 49, 50, 51, 58, 50])


In [3]:
from tokenizer.tokenizer import Tokenizer
tokenizer = Tokenizer("./tokenizer/vocab.txt")

print(res[1])

print(tokenizer.decode(res[1], ignore_special=False))

[2, 2, 5, 323, 5, 296, 5, 5, 75, 5, 243, 278, 278, 5, 394, 5, 5, 5, 51, 5, 5, 247, 5, 5, 360, 5, 364, 5, 5, 57, 5, 5, 5, 238, 122, 5, 65, 5, 5, 167, 271, 5, 5, 142, 5, 68, 5, 5, 5, 3, 3]
['<sos>', '<sos>', '<blk>', 'liao', '<blk>', 'dian', '<blk>', '<blk>', 'pao', '<blk>', 'si', 'ji', 'ji', '<blk>', 'kong', '<blk>', '<blk>', '<blk>', 'cuan', '<blk>', '<blk>', 'hen', '<blk>', '<blk>', 'zong', '<blk>', 'shua', '<blk>', '<blk>', 'qie', '<blk>', '<blk>', '<blk>', 'nian', 'tian', '<blk>', 'wai', '<blk>', '<blk>', 'zhou', 'du', '<blk>', '<blk>', 'jiao', '<blk>', 'die', '<blk>', '<blk>', '<blk>', '<eos>', '<eos>']


**TODO：** 请大家根据CTC的解码思路，将模型的输出进行解码，移除重复字符和blank(上面的版本没有移除重复字符和blank)。

## 评测识别结果

这里我们采用字错率(CER, character error rate)来评测ASR系统的性能，计算公式如下:

$$CER = \frac{S+D+I}{N}$$

pre 代表模型预测， gt 代表正确识别结果。与最小编辑距离一致，将pre转化成gt，其中，S代表将 pre 转化成 gt 需要替换的数量，D 代表将 pre转化成 gt 需要删除的数量，I 代表将 pre 转化成 gt 需要插入的数量，N 代表gt 的长度。


**TODO：** 根据最小编辑距离求出 S，D，I，N ，完成ASR的CER指标评测